In [80]:
import numpy as np
import scipy.integrate as integrate
import scipy.special as special

import random

import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

from matplotlib import pyplot as plt

from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

import csv

In [2]:
# the analytical representation of exact solution
def heat_equ_analytical_solu(x, t):
    return np.sin(np.pi * x) * np.exp(-np.power(np.pi, 2) * t)

In [3]:
# ==========
# helping methods
# ==========

# generate a list from lower and upper bound
def gen_list(p0, pn, delta, dig=5):
    ret = []
    i = p0
    while i < pn:
        ret.append(float(i))
        i += delta
        i = round(i, dig)
    return ret

# padding and zero padding
def padding(origin, a_list, b_list):
    return np.hstack((a_list, origin, b_list))

def zero_padding(origin, num):
    zero_list = [0 for i in range(num)]
    return padding(origin, zero_list, zero_list)

In [4]:
# ==============
# getting trainning data
# ==============

# trainning pairs
def gen_pair(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-1-length)
        train = current_t[p-length:p+length+1]
        solu = next_t[p]
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in average)
def gen_pair_ave(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-2-length)
        train0 = current_t[p-length:p+length+2]
        solu1 = next_t[p]
        solu2 = next_t[p+1]
        train = []
        for j in range(len(train0)-1):
                train.append(0.5 * (train0[j] + train0[j+1]))
        solu = 0.5 * (solu1 + solu2)
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in cell average)
def gen_pair_cell_ave(u, x, t, delta_x, delta_y, length=3, num=1000):
    pairs = []
    for i in range(num):
        t_index = random.randint(0, len(t)-2)
        x_s = random.randint(0, len(x)-2*length-3)
        x_e = x_s+2*length+2
        starting = x[x_s]
        ending = x[x_e]
        input_data = []
        for i in range(x_s, x_e-1):
            value = integrate.quad(lambda x: heat_equ_analytical_solu(x, t[t_index]), x[i], x[i+1])
            value = value[0] * (1/delta_x)
            input_data.append(value)
        target_data = integrate.quad(lambda x: heat_equ_analytical_solu(x, t[t_index+1]), x[x_s+length], x[x_s+length+1])
        target_data = target_data[0] * (1/delta_x)
        pair = {'input': input_data, 'solu': target_data}
        pairs.append(pair)
    return pairs

In [5]:
# restnet
class ResNet(nn.Module):
    def __init__(self, i, h1, h2, o, twolayers):
        super(ResNet, self).__init__()
        self.linear1 = nn.Linear(i, h1)
        self.relu1 = nn.ReLU()
        self.linear21 = nn.Linear(h1, h2)
        self.relu21 = nn.ReLU()
        self.linear22 = nn.Linear(h2, o)
        self.relu22 = nn.ReLU()
        self.linear23 = nn.Linear(h1, o)
        self.relu23 = nn.ReLU()
        self.twolayers = twolayers
        
    def forward(self, x):
        if self.twolayers == True:
            out = self.linear1(x)
            out = self.relu1(out)
            out = self.linear21(out)
            out = self.relu21(out)
            out = self.linear22(out)
            out = self.relu22(out)
        else:
            out = self.linear1(x)
            out = self.relu1(out)
            out = self.linear23(out)
            out = self.relu23(out)
        return out + torch.mean(x)

    def load_model(self, save_path):
        self.load_state_dict(torch.load(save_path))

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)

In [83]:
def calc_next_time(x, t, t_index, delta_x, delta_t, current, model, length=3):
    list_of_error = []
    list_of_predict = []
    padding = np.hstack(([0 for i in range(length)], current, [0 for i in range(length)])).tolist()
    padding_x = np.hstack(([0 for i in range(length)], x, [0 for i in range(length)])).tolist()
    padding_t = np.hstack(([0 for i in range(length)], t, [0 for i in range(length)])).tolist() # not used
    for index in range(len(current)-1):
        j = index + length
        input_data = []
        for i in range(j-length, j+length+1):
            value = integrate.quad(lambda x: heat_equ_analytical_solu(x, t[t_index]), padding_x[i], padding_x[i+1])
            value = value[0] * (1/delta_x)
            input_data.append(value)
        target_data = integrate.quad(lambda x: heat_equ_analytical_solu(x, t[t_index+1]), padding_x[j], padding_x[j+1])
        target_data = target_data[0] * (1/delta_x)
        # print(input_data, target_data)
        tensor_input_data = torch.FloatTensor(input_data)
        out = model(tensor_input_data)
        list_of_predict.append(out.item())
        list_of_error.append(np.abs(out.item()-target_data))
    return list_of_predict, list_of_error

In [109]:
def testing_the_model(delta_x=1/20, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      padding_num=3, hidden=6, twolayers=False, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False,
                      model_name="model_xx delta x=1 20 delta t=1 20", error_file='list_of_error_11 xx.txt', prediction_file='list_of_prediction_11 xx.txt'):
    x = arange(xmin, xmax, delta_x)
    t = arange(ymin, ymax, delta_t)
    print("Generating meshes...")
    X,T = meshgrid(x, t) # grid of point
    print("Generating analytical solution...")
    Z = heat_equ_analytical_solu(X, T) # evaluation of the function on the grid
    print("Preparing the model and the optimizer...")
    model = ResNet(7, hidden, hidden, 1, twolayers=twolayers)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, amsgrad=amsgrad)
    criterion = nn.MSELoss()
    model.train()
    print("Updating data...")
    model.load_model(model_name)
    prediction = []
    error = []
    prediction.append(torch.FloatTensor(Z[0]).tolist())
    for i in range(len(Z)-1):
        list_of_predict, list_of_error = calc_next_time(x, t, i, delta_x, delta_t, Z[i], model)
        prediction.append(list_of_predict)
        error.append(list_of_error)
        print("time", i, "is done", '", the average error is:"', np.average(list_of_error))
        print(len(Z))
        print(len(prediction))
    print("Saving the prediction result...")
    with open(prediction_file, "w") as f:
        writer = csv.writer(f)
        writer.writerows(prediction)
    print("Saving the training error...")
    with open(error_file, "w") as f:
        writer = csv.writer(f)
        writer.writerows(error)

In [ ]:
testing_the_model(delta_x=1/20, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_20_20 iteration=1 layers=1 xx", error_file='./prediction/error pairs_20_20 iteration=1 layers=1 xx.csv', prediction_file='./prediction/prediction pairs_20_20 iteration=1 layers=1 xx.csv')

Generating meshes...
Generating analytical solution...
Preparing the model and the optimizer...
Updating data...
time 0 is done ", the average error is:" 0.21817045333769033
126
2
time 1 is done ", the average error is:" 0.13319262599937787
126
3
time 2 is done ", the average error is:" 0.08131383028285435
126
4
time 3 is done ", the average error is:" 0.04964193492277228
126
5
time 4 is done ", the average error is:" 0.03030630403318991
126
6
time 5 is done ", the average error is:" 0.0185019382441407
126
7
time 6 is done ", the average error is:" 0.01129539641203453
126
8
time 7 is done ", the average error is:" 0.006895817382401729
126
9
time 8 is done ", the average error is:" 0.004209882948294138
126
10
time 9 is done ", the average error is:" 0.002570125272639946
126
11
time 10 is done ", the average error is:" 0.0015690564059091688
126
12
time 11 is done ", the average error is:" 0.0009579058386086275
126
13
time 12 is done ", the average error is:" 0.0005847996118801145
126
14


In [95]:
testing_the_model(delta_x=1/20, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_20_20 iteration=1 layers=1 xx", error_file='./prediction/error pairs_20_20 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_20_20 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_20_40 iteration=1 layers=1 xx", error_file='./prediction/error pairs_20_40 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_20_40 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_20_80 iteration=1 layers=1 xx", error_file='./prediction/error pairs_20_80 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_20_80 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_20_160 iteration=1 layers=1 xx", error_file='./prediction/error pairs_20_160 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_20_160 iteration=1 layers=1 xx')

Generating meshes...
Generating analytical solution...
Preparing the model and the optimizer...
Updating data...
time 0 is done ", the average error is:" 0.21817045333769033
time 1 is done ", the average error is:" 0.13319262599937787
time 2 is done ", the average error is:" 0.08131383028285435
time 3 is done ", the average error is:" 0.04964193492277228
time 4 is done ", the average error is:" 0.03030630403318991
time 5 is done ", the average error is:" 0.0185019382441407
time 6 is done ", the average error is:" 0.01129539641203453
time 7 is done ", the average error is:" 0.006895817382401729
time 8 is done ", the average error is:" 0.004209882948294138
time 9 is done ", the average error is:" 0.002570125272639946
time 10 is done ", the average error is:" 0.0015690564059091688
time 11 is done ", the average error is:" 0.0009579058386086275
time 12 is done ", the average error is:" 0.0005847996118801145
time 13 is done ", the average error is:" 0.00035701900102048614
time 14 is done ",

time 0 is done ", the average error is:" 0.11545401844815538
time 1 is done ", the average error is:" 0.09020927463259412
time 2 is done ", the average error is:" 0.07048444533697394
time 3 is done ", the average error is:" 0.05507257445894353
time 4 is done ", the average error is:" 0.04303060982044588
time 5 is done ", the average error is:" 0.03362170275970886
time 6 is done ", the average error is:" 0.026270104429656785
time 7 is done ", the average error is:" 0.020525979643962695
time 8 is done ", the average error is:" 0.01603784767029593
time 9 is done ", the average error is:" 0.012531070231735596
time 10 is done ", the average error is:" 0.0097910738110027
time 11 is done ", the average error is:" 0.007650193273567845
time 12 is done ", the average error is:" 0.005977430877245727
time 13 is done ", the average error is:" 0.004670428697728565
time 14 is done ", the average error is:" 0.003649209924981598
time 15 is done ", the average error is:" 0.0028512879397928396
time 16 is

time 128 is done ", the average error is:" 2.219227625784056e-15
time 129 is done ", the average error is:" 1.7339799146886382e-15
time 130 is done ", the average error is:" 1.3548342378634761e-15
time 131 is done ", the average error is:" 1.0585911226473303e-15
time 132 is done ", the average error is:" 8.271236458778302e-16
time 133 is done ", the average error is:" 6.46267826876691e-16
time 134 is done ", the average error is:" 5.049574375142409e-16
time 135 is done ", the average error is:" 3.9454519453635647e-16
time 136 is done ", the average error is:" 3.0827549325662457e-16
time 137 is done ", the average error is:" 2.408690739049988e-16
time 138 is done ", the average error is:" 1.8820153317836896e-16
time 139 is done ", the average error is:" 1.4705012933696504e-16
time 140 is done ", the average error is:" 1.1489668627678733e-16
time 141 is done ", the average error is:" 8.977380725040217e-17
time 142 is done ", the average error is:" 7.014419772098845e-17
time 143 is done "

time 0 is done ", the average error is:" 0.2906147857694572
time 1 is done ", the average error is:" 0.26936604469695097
time 2 is done ", the average error is:" 0.24822770971338426
time 3 is done ", the average error is:" 0.22884463892710696
time 4 is done ", the average error is:" 0.21146252701483853
time 5 is done ", the average error is:" 0.1953871681717756
time 6 is done ", the average error is:" 0.1798051439675549
time 7 is done ", the average error is:" 0.16613533357443802
time 8 is done ", the average error is:" 0.1544877393441882
time 9 is done ", the average error is:" 0.14432526448972088
time 10 is done ", the average error is:" 0.1356307763741245
time 11 is done ", the average error is:" 0.12810733358692636
time 12 is done ", the average error is:" 0.12209322395044826
time 13 is done ", the average error is:" 0.11731025180216273
time 14 is done ", the average error is:" 0.11376164121473266
time 15 is done ", the average error is:" 0.1114674803224154
time 16 is done ", the a

time 134 is done ", the average error is:" 0.11428946217119916
time 135 is done ", the average error is:" 0.11428946292805922
time 136 is done ", the average error is:" 0.11428946337514939
time 137 is done ", the average error is:" 0.11428946357343768
time 138 is done ", the average error is:" 0.11428946644475678
time 139 is done ", the average error is:" 0.11428946746021898
time 140 is done ", the average error is:" 0.11428946894534701
time 141 is done ", the average error is:" 0.11428947078176392
time 142 is done ", the average error is:" 0.11428947070522893
time 143 is done ", the average error is:" 0.11428947181583392
time 144 is done ", the average error is:" 0.11428947190878187
time 145 is done ", the average error is:" 0.11428947122299948
time 146 is done ", the average error is:" 0.11428947232193566
time 147 is done ", the average error is:" 0.11428947192773198
time 148 is done ", the average error is:" 0.114289471948088
time 149 is done ", the average error is:" 0.114289471966

...

time 402 is done ", the average error is:" 3.39440261975936e-13
time 403 is done ", the average error is:" 3.191347205526206e-13
time 404 is done ", the average error is:" 3.00043563910828e-13
time 405 is done ", the average error is:" 2.8209441221766387e-13
time 406 is done ", the average error is:" 2.6521949539981523e-13
time 407 is done ", the average error is:" 2.4935379002103386e-13
time 408 is done ", the average error is:" 2.3443731131704486e-13
time 409 is done ", the average error is:" 2.2041295147458652e-13
time 410 is done ", the average error is:" 2.072274898409915e-13
time 411 is done ", the average error is:" 1.9483081724047799e-13
time 412 is done ", the average error is:" 1.8317623083840212e-13
time 413 is done ", the average error is:" 1.7221815499933156e-13
time 414 is done ", the average error is:" 1.6191582962287318e-13
time 415 is done ", the average error is:" 1.5223002501624598e-13
time 416 is done ", the average error is:" 1.4312356702394067e-13
time 417 is done

time 528 is done ", the average error is:" 1.4298499568046463e-16
time 529 is done ", the average error is:" 1.3443146698826598e-16
time 530 is done ", the average error is:" 1.2638962620807494e-16
time 531 is done ", the average error is:" 1.188288061603345e-16
time 532 is done ", the average error is:" 1.117203817197879e-16
time 533 is done ", the average error is:" 1.0503716109588264e-16
time 534 is done ", the average error is:" 9.875369767530697e-17
time 535 is done ", the average error is:" 9.284622517967982e-17
time 536 is done ", the average error is:" 8.729214256074025e-17
time 537 is done ", the average error is:" 8.207017780028016e-17
time 538 is done ", the average error is:" 7.716062379841664e-17
time 539 is done ", the average error is:" 7.254472228217356e-17
time 540 is done ", the average error is:" 6.820503485717774e-17
time 541 is done ", the average error is:" 6.412497906702028e-17
time 542 is done ", the average error is:" 6.028891972260088e-17
time 543 is done ", t

time 654 is done ", the average error is:" 6.02306440694565e-20
time 655 is done ", the average error is:" 5.66275310841619e-20
time 656 is done ", the average error is:" 5.324006437006969e-20
time 657 is done ", the average error is:" 5.0055214503781416e-20
time 658 is done ", the average error is:" 4.706083380673011e-20
time 659 is done ", the average error is:" 4.4245636429058497e-20
time 660 is done ", the average error is:" 4.1598788420679854e-20
time 661 is done ", the average error is:" 3.911034513829589e-20
time 662 is done ", the average error is:" 3.6770661706744926e-20
time 663 is done ", the average error is:" 3.457102326183617e-20
time 664 is done ", the average error is:" 3.2502955849394805e-20
time 665 is done ", the average error is:" 3.0558537071891194e-20
time 666 is done ", the average error is:" 2.873054209740013e-20
time 667 is done ", the average error is:" 2.701185621021647e-20
time 668 is done ", the average error is:" 2.5395975961370404e-20
time 669 is done ", 

time 780 is done ", the average error is:" 2.537143120081565e-23
time 781 is done ", the average error is:" 2.3853651316682505e-23
time 782 is done ", the average error is:" 2.24267172858226e-23
time 783 is done ", the average error is:" 2.1085134973087507e-23
time 784 is done ", the average error is:" 1.9823813498716115e-23
time 785 is done ", the average error is:" 1.8637931428713215e-23
time 786 is done ", the average error is:" 1.7522968109203234e-23
time 787 is done ", the average error is:" 1.6474756774991064e-23
time 788 is done ", the average error is:" 1.548919006338665e-23
time 789 is done ", the average error is:" 1.4562616553481385e-23
time 790 is done ", the average error is:" 1.3691456099799396e-23
time 791 is done ", the average error is:" 1.2872431101746383e-23
time 792 is done ", the average error is:" 1.2102399025579049e-23
time 793 is done ", the average error is:" 1.1378411791555155e-23
time 794 is done ", the average error is:" 1.0697732745149441e-23
time 795 is do

time 906 is done ", the average error is:" 1.0687392179174957e-26
time 907 is done ", the average error is:" 1.004807402481877e-26
time 908 is done ", the average error is:" 9.446977099102496e-27
time 909 is done ", the average error is:" 8.881847486628678e-27
time 910 is done ", the average error is:" 8.350535208279395e-27
time 911 is done ", the average error is:" 7.850985058589873e-27
time 912 is done ", the average error is:" 7.381335931706528e-27
time 913 is done ", the average error is:" 6.939778256173353e-27
time 914 is done ", the average error is:" 6.524629854872654e-27
time 915 is done ", the average error is:" 6.1343191909390556e-27
time 916 is done ", the average error is:" 5.767362620510801e-27
time 917 is done ", the average error is:" 5.422348313215982e-27
time 918 is done ", the average error is:" 5.097978433119658e-27
time 919 is done ", the average error is:" 4.793017366951998e-27
time 920 is done ", the average error is:" 4.5062907612746056e-27
time 921 is done ", th

In [96]:
testing_the_model(delta_x=1/20, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_40_20 iteration=1 layers=1 xx", error_file='./prediction/error pairs_40_20 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_40_20 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_40_40 iteration=1 layers=1 xx", error_file='./prediction/error pairs_40_40 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_40_40 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_40_80 iteration=1 layers=1 xx", error_file='./prediction/error pairs_40_80 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_40_80 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_40_160 iteration=1 layers=1 xx", error_file='./prediction/error pairs_40_160 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_40_160 iteration=1 layers=1 xx')

Generating meshes...
Generating analytical solution...
Preparing the model and the optimizer...
Updating data...
time 0 is done ", the average error is:" 0.2788523043159346
time 1 is done ", the average error is:" 0.1782025261761541
time 2 is done ", the average error is:" 0.11090555034960087
time 3 is done ", the average error is:" 0.06804094786699146
time 4 is done ", the average error is:" 0.04188583969666226
time 5 is done ", the average error is:" 0.025821165981363074
time 6 is done ", the average error is:" 0.016043641937194212
time 7 is done ", the average error is:" 0.010110545812125693
time 8 is done ", the average error is:" 0.006576059241511583
time 9 is done ", the average error is:" 0.004574433878058596
time 10 is done ", the average error is:" 0.0036825895105813628
time 11 is done ", the average error is:" 0.003582584631518838
time 12 is done ", the average error is:" 0.003594141415522629
time 13 is done ", the average error is:" 0.0036011968674220214
time 14 is done ", t

time 1 is done ", the average error is:" 0.09020927463259412
time 2 is done ", the average error is:" 0.07048444533697394
time 3 is done ", the average error is:" 0.05507257445894353
time 4 is done ", the average error is:" 0.04303060982044588
time 5 is done ", the average error is:" 0.03362170275970886
time 6 is done ", the average error is:" 0.026270104429656785
time 7 is done ", the average error is:" 0.020525979643962695
time 8 is done ", the average error is:" 0.01603784767029593
time 9 is done ", the average error is:" 0.012531070231735596
time 10 is done ", the average error is:" 0.0097910738110027
time 11 is done ", the average error is:" 0.007650193273567845
time 12 is done ", the average error is:" 0.005977430877245727
time 13 is done ", the average error is:" 0.004670428697728565
time 14 is done ", the average error is:" 0.003649209924981598
time 15 is done ", the average error is:" 0.0028512879397928396
time 16 is done ", the average error is:" 0.0022278355067259403
time 17

time 129 is done ", the average error is:" 1.7339799146886382e-15
time 130 is done ", the average error is:" 1.3548342378634761e-15
time 131 is done ", the average error is:" 1.0585911226473303e-15
time 132 is done ", the average error is:" 8.271236458778302e-16
time 133 is done ", the average error is:" 6.46267826876691e-16
time 134 is done ", the average error is:" 5.049574375142409e-16
time 135 is done ", the average error is:" 3.9454519453635647e-16
time 136 is done ", the average error is:" 3.0827549325662457e-16
time 137 is done ", the average error is:" 2.408690739049988e-16
time 138 is done ", the average error is:" 1.8820153317836896e-16
time 139 is done ", the average error is:" 1.4705012933696504e-16
time 140 is done ", the average error is:" 1.1489668627678733e-16
time 141 is done ", the average error is:" 8.977380725040217e-17
time 142 is done ", the average error is:" 7.014419772098845e-17
time 143 is done ", the average error is:" 5.480673891240834e-17
time 144 is done "

time 1 is done ", the average error is:" 0.19731768218853474
time 2 is done ", the average error is:" 0.1895323336897341
time 3 is done ", the average error is:" 0.18094473704156763
time 4 is done ", the average error is:" 0.17085875616274962
time 5 is done ", the average error is:" 0.15829223362259293
time 6 is done ", the average error is:" 0.1407966997663537
time 7 is done ", the average error is:" 0.12435546053224666
time 8 is done ", the average error is:" 0.10984728025775586
time 9 is done ", the average error is:" 0.09704764494058082
time 10 is done ", the average error is:" 0.08575671419126181
time 11 is done ", the average error is:" 0.07579073748374757
time 12 is done ", the average error is:" 0.06697781693970795
time 13 is done ", the average error is:" 0.05918445669735894
time 14 is done ", the average error is:" 0.05229561959547282
time 15 is done ", the average error is:" 0.04620632775630087
time 16 is done ", the average error is:" 0.04082377211000755
time 17 is done ", 

time 130 is done ", the average error is:" 5.8250746004583075e-09
time 131 is done ", the average error is:" 5.148996367252507e-09
time 132 is done ", the average error is:" 4.5513855938925e-09
time 133 is done ", the average error is:" 4.023136834830763e-09
time 134 is done ", the average error is:" 3.5561980415375775e-09
time 135 is done ", the average error is:" 3.143453866923431e-09
time 136 is done ", the average error is:" 2.7786121388465564e-09
time 137 is done ", the average error is:" 2.4561164481678372e-09
time 138 is done ", the average error is:" 2.1710505896479906e-09
time 139 is done ", the average error is:" 1.9190713792355503e-09
time 140 is done ", the average error is:" 1.6963376381463958e-09
time 141 is done ", the average error is:" 1.4994541736734868e-09
time 142 is done ", the average error is:" 1.3254226344468625e-09
time 143 is done ", the average error is:" 1.1715891343190848e-09
time 144 is done ", the average error is:" 1.0356107709030354e-09
time 145 is done

...

time 394 is done ", the average error is:" 0.007987629621830297
time 395 is done ", the average error is:" 0.007987629621853669
time 396 is done ", the average error is:" 0.007987629621875643
time 397 is done ", the average error is:" 0.0079876296218963
time 398 is done ", the average error is:" 0.007987629621915725
time 399 is done ", the average error is:" 0.007987629621933985
time 400 is done ", the average error is:" 0.007987629621951155
time 401 is done ", the average error is:" 0.007987629621967297
time 402 is done ", the average error is:" 0.007987629621982474
time 403 is done ", the average error is:" 0.007987629621996742
time 404 is done ", the average error is:" 0.007987629622010156
time 405 is done ", the average error is:" 0.00798762962202277
time 406 is done ", the average error is:" 0.007987629622034628
time 407 is done ", the average error is:" 0.007987629622045777
time 408 is done ", the average error is:" 0.007987629622056258
time 409 is done ", the average error is:" 

time 523 is done ", the average error is:" 0.007987629622220856
time 524 is done ", the average error is:" 0.007987629622220865
time 525 is done ", the average error is:" 0.007987629622220872
time 526 is done ", the average error is:" 0.007987629622220879
time 527 is done ", the average error is:" 0.007987629622220885
time 528 is done ", the average error is:" 0.007987629622220892
time 529 is done ", the average error is:" 0.0079876296222209
time 530 is done ", the average error is:" 0.007987629622220905
time 531 is done ", the average error is:" 0.00798762962222091
time 532 is done ", the average error is:" 0.007987629622220913
time 533 is done ", the average error is:" 0.00798762962222092
time 534 is done ", the average error is:" 0.007987629622220924
time 535 is done ", the average error is:" 0.007987629622220927
time 536 is done ", the average error is:" 0.007987629622220932
time 537 is done ", the average error is:" 0.007987629622220936
time 538 is done ", the average error is:" 0

time 652 is done ", the average error is:" 0.007987629622220993
time 653 is done ", the average error is:" 0.007987629622220993
time 654 is done ", the average error is:" 0.007987629622220993
time 655 is done ", the average error is:" 0.007987629622220993
time 656 is done ", the average error is:" 0.007987629622220993
time 657 is done ", the average error is:" 0.007987629622220993
time 658 is done ", the average error is:" 0.007987629622220993
time 659 is done ", the average error is:" 0.007987629622220993
time 660 is done ", the average error is:" 0.007987629622220993
time 661 is done ", the average error is:" 0.007987629622220993
time 662 is done ", the average error is:" 0.007987629622220993
time 663 is done ", the average error is:" 0.007987629622220993
time 664 is done ", the average error is:" 0.007987629622220993
time 665 is done ", the average error is:" 0.007987629622220993
time 666 is done ", the average error is:" 0.007987629622220993
time 667 is done ", the average error is

time 781 is done ", the average error is:" 0.007987629622220993
time 782 is done ", the average error is:" 0.007987629622220993
time 783 is done ", the average error is:" 0.007987629622220993
time 784 is done ", the average error is:" 0.007987629622220993
time 785 is done ", the average error is:" 0.007987629622220993
time 786 is done ", the average error is:" 0.007987629622220993
time 787 is done ", the average error is:" 0.007987629622220993
time 788 is done ", the average error is:" 0.007987629622220993
time 789 is done ", the average error is:" 0.007987629622220993
time 790 is done ", the average error is:" 0.007987629622220993
time 791 is done ", the average error is:" 0.007987629622220993
time 792 is done ", the average error is:" 0.007987629622220993
time 793 is done ", the average error is:" 0.007987629622220993
time 794 is done ", the average error is:" 0.007987629622220993
time 795 is done ", the average error is:" 0.007987629622220993
time 796 is done ", the average error is

time 911 is done ", the average error is:" 0.007987629622220993
time 912 is done ", the average error is:" 0.007987629622220993
time 913 is done ", the average error is:" 0.007987629622220993
time 914 is done ", the average error is:" 0.007987629622220993
time 915 is done ", the average error is:" 0.007987629622220993
time 916 is done ", the average error is:" 0.007987629622220993
time 917 is done ", the average error is:" 0.007987629622220993
time 918 is done ", the average error is:" 0.007987629622220993
time 919 is done ", the average error is:" 0.007987629622220993
time 920 is done ", the average error is:" 0.007987629622220993
time 921 is done ", the average error is:" 0.007987629622220993
time 922 is done ", the average error is:" 0.007987629622220993
time 923 is done ", the average error is:" 0.007987629622220993
time 924 is done ", the average error is:" 0.007987629622220993
time 925 is done ", the average error is:" 0.007987629622220993
time 926 is done ", the average error is

In [97]:
testing_the_model(delta_x=1/80, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_80_20 iteration=1 layers=1 xx", error_file='./prediction/error pairs_80_20 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_80_20 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/80, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_80_40 iteration=1 layers=1 xx", error_file='./prediction/error pairs_80_40 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_80_40 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/80, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_80_80 iteration=1 layers=1 xx", error_file='./prediction/error pairs_80_80 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_80_80 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/80, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_80_160 iteration=1 layers=1 xx", error_file='./prediction/error pairs_80_160 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_80_160 iteration=1 layers=1 xx')

Generating meshes...
Generating analytical solution...
Preparing the model and the optimizer...
Updating data...
time 0 is done ", the average error is:" 0.2620677542651781
time 1 is done ", the average error is:" 0.17568763619945416
time 2 is done ", the average error is:" 0.11762166743577046
time 3 is done ", the average error is:" 0.07334512393191363
time 4 is done ", the average error is:" 0.045998309527433646
time 5 is done ", the average error is:" 0.02932181910970902
time 6 is done ", the average error is:" 0.01917985519864795
time 7 is done ", the average error is:" 0.01305236845723374
time 8 is done ", the average error is:" 0.009409907371252498
time 9 is done ", the average error is:" 0.007351556458351815
time 10 is done ", the average error is:" 0.0064048233963017375
time 11 is done ", the average error is:" 0.006362195526321184
time 12 is done ", the average error is:" 0.0063808882024263625
time 13 is done ", the average error is:" 0.006392300617306475
time 14 is done ", th

time 1 is done ", the average error is:" 0.24177379485866823
time 2 is done ", the average error is:" 0.18905708745610017
time 3 is done ", the average error is:" 0.1478628012988249
time 4 is done ", the average error is:" 0.11571980709871985
time 5 is done ", the average error is:" 0.09064518013321216
time 6 is done ", the average error is:" 0.07104928079544622
time 7 is done ", the average error is:" 0.05577136318181298
time 8 is done ", the average error is:" 0.04382789856974335
time 9 is done ", the average error is:" 0.03449148011406042
time 10 is done ", the average error is:" 0.02718929483933955
time 11 is done ", the average error is:" 0.02147919936179343
time 12 is done ", the average error is:" 0.017014097634759102
time 13 is done ", the average error is:" 0.01352254691754503
time 14 is done ", the average error is:" 0.010788394750007122
time 15 is done ", the average error is:" 0.008638072925761402
time 16 is done ", the average error is:" 0.006921755902524066
time 17 is don

time 131 is done ", the average error is:" 0.0038478691130875266
time 132 is done ", the average error is:" 0.003847869113087554
time 133 is done ", the average error is:" 0.003847869113087576
time 134 is done ", the average error is:" 0.003847869113087593
time 135 is done ", the average error is:" 0.0038478691130876064
time 136 is done ", the average error is:" 0.003847869113087617
time 137 is done ", the average error is:" 0.003847869113087625
time 138 is done ", the average error is:" 0.0038478691130876316
time 139 is done ", the average error is:" 0.0038478691130876363
time 140 is done ", the average error is:" 0.0038478691130876407
time 141 is done ", the average error is:" 0.0038478691130876437
time 142 is done ", the average error is:" 0.003847869113087646
time 143 is done ", the average error is:" 0.003847869113087648
time 144 is done ", the average error is:" 0.003847869113087649
time 145 is done ", the average error is:" 0.00384786911308765
time 146 is done ", the average err

time 6 is done ", the average error is:" 0.13689756820577395
time 7 is done ", the average error is:" 0.12245351277911729
time 8 is done ", the average error is:" 0.10962270190230074
time 9 is done ", the average error is:" 0.0982105130999569
time 10 is done ", the average error is:" 0.0880566698742224
time 11 is done ", the average error is:" 0.079140110856077
time 12 is done ", the average error is:" 0.07061558717056333
time 13 is done ", the average error is:" 0.06244149758049842
time 14 is done ", the average error is:" 0.05521824428598246
time 15 is done ", the average error is:" 0.04884844818971203
time 16 is done ", the average error is:" 0.04321795255957642
time 17 is done ", the average error is:" 0.038240951530245836
time 18 is done ", the average error is:" 0.033841598449854016
time 19 is done ", the average error is:" 0.029952850692406868
time 20 is done ", the average error is:" 0.026514948001627976
time 21 is done ", the average error is:" 0.02346321151720843
time 22 is d

time 134 is done ", the average error is:" 0.0002241864242815749
time 135 is done ", the average error is:" 0.0002241860021628152
time 136 is done ", the average error is:" 0.00022418621209381866
time 137 is done ", the average error is:" 0.00022418673561567422
time 138 is done ", the average error is:" 0.00022418680923284875
time 139 is done ", the average error is:" 0.00022418665600148135
time 140 is done ", the average error is:" 0.00022418656576333752
time 141 is done ", the average error is:" 0.00022418642031246978
time 142 is done ", the average error is:" 0.0002241865166950887
time 143 is done ", the average error is:" 0.00022418664030651145
time 144 is done ", the average error is:" 0.00022418685064842304
time 145 is done ", the average error is:" 0.0002241870441731171
time 146 is done ", the average error is:" 0.00022418727292749277
time 147 is done ", the average error is:" 0.00022418750019339456
time 148 is done ", the average error is:" 0.00022418782990489943
time 149 is do

...

time 389 is done ", the average error is:" 1.6725797432141085e-12
time 390 is done ", the average error is:" 1.5725243953446596e-12
time 391 is done ", the average error is:" 1.4784543895751126e-12
time 392 is done ", the average error is:" 1.3900117082930848e-12
time 393 is done ", the average error is:" 1.3068596085432974e-12
time 394 is done ", the average error is:" 1.2286820649658478e-12
time 395 is done ", the average error is:" 1.155181030827248e-12
time 396 is done ", the average error is:" 1.0860769760881133e-12
time 397 is done ", the average error is:" 1.021106553096125e-12
time 398 is done ", the average error is:" 9.600227092288159e-13
time 399 is done ", the average error is:" 9.025932296228391e-13
time 400 is done ", the average error is:" 8.485992562723985e-13
time 401 is done ", the average error is:" 7.978352607587575e-13
time 402 is done ", the average error is:" 7.501079877732585e-13
time 403 is done ", the average error is:" 7.052357998631102e-13
time 404 is done "

time 515 is done ", the average error is:" 7.04553551733115e-16
time 516 is done ", the average error is:" 6.624063640207535e-16
time 517 is done ", the average error is:" 6.227805055572103e-16
time 518 is done ", the average error is:" 5.855252108311003e-16
time 519 is done ", the average error is:" 5.504984880374808e-16
time 520 is done ", the average error is:" 5.175671158563585e-16
time 521 is done ", the average error is:" 4.866056722191755e-16
time 522 is done ", the average error is:" 4.574965473145489e-16
time 523 is done ", the average error is:" 4.3012853042810507e-16
time 524 is done ", the average error is:" 4.043977561889207e-16
time 525 is done ", the average error is:" 3.802062528854026e-16
time 526 is done ", the average error is:" 3.5746193634474406e-16
time 527 is done ", the average error is:" 3.360781762674814e-16
time 528 is done ", the average error is:" 3.159737019661786e-16
time 529 is done ", the average error is:" 2.970717674310324e-16
time 530 is done ", the 

time 641 is done ", the average error is:" 2.9678439823519457e-19
time 642 is done ", the average error is:" 2.790304529085939e-19
time 643 is done ", the average error is:" 2.6233861013629333e-19
time 644 is done ", the average error is:" 2.4664520077473257e-19
time 645 is done ", the average error is:" 2.3189068473429384e-19
time 646 is done ", the average error is:" 2.1801872168095624e-19
time 647 is done ", the average error is:" 2.049765788280359e-19
time 648 is done ", the average error is:" 1.9271468640014602e-19
time 649 is done ", the average error is:" 1.8118630632002526e-19
time 650 is done ", the average error is:" 1.703475997470536e-19
time 651 is done ", the average error is:" 1.601571861612365e-19
time 652 is done ", the average error is:" 1.5057643142297344e-19
time 653 is done ", the average error is:" 1.4156874172340764e-19
time 654 is done ", the average error is:" 1.3309997564760382e-19
time 655 is done ", the average error is:" 1.2513780586643186e-19
time 656 is do

time 767 is done ", the average error is:" 1.2501675924187628e-22
time 768 is done ", the average error is:" 1.1753809531093783e-22
time 769 is done ", the average error is:" 1.1050687446807926e-22
time 770 is done ", the average error is:" 1.038962402688467e-22
time 771 is done ", the average error is:" 9.768105639981697e-23
time 772 is done ", the average error is:" 9.183766977351959e-23
time 773 is done ", the average error is:" 8.634387954939091e-23
time 774 is done ", the average error is:" 8.117868628587311e-23
time 775 is done ", the average error is:" 7.632247556420195e-23
time 776 is done ", the average error is:" 7.175676661991317e-23
time 777 is done ", the average error is:" 6.746423176325627e-23
time 778 is done ", the average error is:" 6.342843740998707e-23
time 779 is done ", the average error is:" 5.96340970077366e-23
time 780 is done ", the average error is:" 5.60667397246746e-23
time 781 is done ", the average error is:" 5.2712746532764e-23
time 782 is done ", the av

time 893 is done ", the average error is:" 5.266177219233875e-26
time 894 is done ", the average error is:" 4.951147823147803e-26
time 895 is done ", the average error is:" 4.654965889037484e-26
time 896 is done ", the average error is:" 4.376500714544924e-26
time 897 is done ", the average error is:" 4.1146935744073847e-26
time 898 is done ", the average error is:" 3.868549377610952e-26
time 899 is done ", the average error is:" 3.6371282733603276e-26
time 900 is done ", the average error is:" 3.419551439488127e-26
time 901 is done ", the average error is:" 3.214990049517239e-26
time 902 is done ", the average error is:" 3.0226661608166487e-26
time 903 is done ", the average error is:" 2.8418473697355585e-26
time 904 is done ", the average error is:" 2.67184461724413e-26
time 905 is done ", the average error is:" 2.512012239085347e-26
time 906 is done ", the average error is:" 2.3617419723114207e-26
time 907 is done ", the average error is:" 2.2204593515712207e-26
time 908 is done ", 

In [98]:
testing_the_model(delta_x=1/160, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_160_20 iteration=1 layers=1 xx", error_file='./prediction/error pairs_160_20 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_160_20 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/160, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_160_40 iteration=1 layers=1 xx", error_file='./prediction/error pairs_160_40 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_160_40 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/160, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_160_80 iteration=1 layers=1 xx", error_file='./prediction/error pairs_160_80 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_160_80 iteration=1 layers=1 xx')
testing_the_model(delta_x=1/160, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_160_160 iteration=1 layers=1 xx", error_file='./prediction/error pairs_160_160 iteration=1 layers=1 xx', prediction_file='./prediction/prediction pairs_160_160 iteration=1 layers=1 xx')

Generating meshes...
Generating analytical solution...
Preparing the model and the optimizer...
Updating data...
time 0 is done ", the average error is:" 0.15833613499270588
time 1 is done ", the average error is:" 0.09838487035164778
time 2 is done ", the average error is:" 0.06124612482661318
time 3 is done ", the average error is:" 0.03705730186771472
time 4 is done ", the average error is:" 0.022297346424919588
time 5 is done ", the average error is:" 0.013327526323550148
time 6 is done ", the average error is:" 0.00790433104500832
time 7 is done ", the average error is:" 0.004649371889149154
time 8 is done ", the average error is:" 0.0027687364090070287
time 9 is done ", the average error is:" 0.0018954569816961601
time 10 is done ", the average error is:" 0.001410042098894406
time 11 is done ", the average error is:" 0.0010457924013240493
time 12 is done ", the average error is:" 0.0006704452802439934
time 13 is done ", the average error is:" 0.0004098860954659527
time 14 is done

Generating meshes...
Generating analytical solution...
Preparing the model and the optimizer...
Updating data...
time 0 is done ", the average error is:" 0.14336634121103295
time 1 is done ", the average error is:" 0.11210465924699703
time 2 is done ", the average error is:" 0.08767855239174774
time 3 is done ", the average error is:" 0.06859336314067832
time 4 is done ", the average error is:" 0.05368126675019237
time 5 is done ", the average error is:" 0.04198906279799849
time 6 is done ", the average error is:" 0.032807889143370655
time 7 is done ", the average error is:" 0.02563423775780514
time 8 is done ", the average error is:" 0.0200291518215673
time 9 is done ", the average error is:" 0.015649652370553727
time 10 is done ", the average error is:" 0.012227757572334156
time 11 is done ", the average error is:" 0.009554081634539984
time 12 is done ", the average error is:" 0.007465022069491069
time 13 is done ", the average error is:" 0.005832748065871769
time 14 is done ", the a

time 127 is done ", the average error is:" 3.5471228709262685e-15
time 128 is done ", the average error is:" 2.7715221470185343e-15
time 129 is done ", the average error is:" 2.165511518799893e-15
time 130 is done ", the average error is:" 1.6920087714971305e-15
time 131 is done ", the average error is:" 1.3220406300504817e-15
time 132 is done ", the average error is:" 1.032968057959681e-15
time 133 is done ", the average error is:" 8.071031276734501e-16
time 134 is done ", the average error is:" 6.306249704695939e-16
time 135 is done ", the average error is:" 4.927348744687997e-16
time 136 is done ", the average error is:" 3.8499529235943746e-16
time 137 is done ", the average error is:" 3.0081365027432775e-16
time 138 is done ", the average error is:" 2.350388743205087e-16
time 139 is done ", the average error is:" 1.8364614258673342e-16
time 140 is done ", the average error is:" 1.4349076488597245e-16
time 141 is done ", the average error is:" 1.1211561516029923e-16
time 142 is done

time 0 is done ", the average error is:" 0.0803596167651741
time 1 is done ", the average error is:" 0.07103280123499423
time 2 is done ", the average error is:" 0.06278847579670083
time 3 is done ", the average error is:" 0.055501033184339035
time 4 is done ", the average error is:" 0.04905938668565404
time 5 is done ", the average error is:" 0.043365385915888295
time 6 is done ", the average error is:" 0.03833224537366358
time 7 is done ", the average error is:" 0.033883270673408517
time 8 is done ", the average error is:" 0.02995065740589183
time 9 is done ", the average error is:" 0.026474481891819397
time 10 is done ", the average error is:" 0.023401761538915936
time 11 is done ", the average error is:" 0.02068566944631521
time 12 is done ", the average error is:" 0.01828481783678988
time 13 is done ", the average error is:" 0.01616261849804876
time 14 is done ", the average error is:" 0.014286727044114792
time 15 is done ", the average error is:" 0.01262856079469341
time 16 is do

time 128 is done ", the average error is:" 1.114125910350133e-08
time 129 is done ", the average error is:" 9.8481652292668e-09
time 130 is done ", the average error is:" 8.705153542080078e-09
time 131 is done ", the average error is:" 7.69480286737456e-09
time 132 is done ", the average error is:" 6.80171665900728e-09
time 133 is done ", the average error is:" 6.012286312080424e-09
time 134 is done ", the average error is:" 5.314478779542268e-09
time 135 is done ", the average error is:" 4.697661704076883e-09
time 136 is done ", the average error is:" 4.152434770741815e-09
time 137 is done ", the average error is:" 3.6704888330324133e-09
time 138 is done ", the average error is:" 3.2444788997730022e-09
time 139 is done ", the average error is:" 2.8679133451069637e-09
time 140 is done ", the average error is:" 2.535053277392137e-09
time 141 is done ", the average error is:" 2.2408261137891877e-09
time 142 is done ", the average error is:" 1.9807479631870423e-09
time 143 is done ", the 

...

time 380 is done ", the average error is:" 3.5078652673790024e-22
time 381 is done ", the average error is:" 3.1007301349247996e-22
time 382 is done ", the average error is:" 2.740848344617794e-22
time 383 is done ", the average error is:" 2.4227361721725396e-22
time 384 is done ", the average error is:" 2.1415447109014555e-22
time 385 is done ", the average error is:" 1.8929895855359758e-22
time 386 is done ", the average error is:" 1.6732825206553108e-22
time 387 is done ", the average error is:" 1.4790754475032507e-22
time 388 is done ", the average error is:" 1.3074087601851367e-22
time 389 is done ", the average error is:" 1.1556663240136553e-22
time 390 is done ", the average error is:" 1.02153566306825e-22
time 391 is done ", the average error is:" 9.029727473212749e-23
time 392 is done ", the average error is:" 7.981705661073139e-23
time 393 is done ", the average error is:" 7.055320862470383e-23
time 394 is done ", the average error is:" 6.236455739106201e-23
time 395 is done 

time 1 is done ", the average error is:" 0.045753045610982264
time 2 is done ", the average error is:" 0.04338114477557528
time 3 is done ", the average error is:" 0.04125032405253297
time 4 is done ", the average error is:" 0.03947054578445551
time 5 is done ", the average error is:" 0.03766840458339761
time 6 is done ", the average error is:" 0.03575252956072488
time 7 is done ", the average error is:" 0.03391217576985086
time 8 is done ", the average error is:" 0.03213910212151141
time 9 is done ", the average error is:" 0.03042190498129975
time 10 is done ", the average error is:" 0.028745926760935396
time 11 is done ", the average error is:" 0.027089382819876554
time 12 is done ", the average error is:" 0.02538528503570669
time 13 is done ", the average error is:" 0.02366650121097537
time 14 is done ", the average error is:" 0.022051243437968444
time 15 is done ", the average error is:" 0.02053225469050932
time 16 is done ", the average error is:" 0.019105193383979053
time 17 is d

time 130 is done ", the average error is:" 1.4932314767169017e-05
time 131 is done ", the average error is:" 1.4039048408647801e-05
time 132 is done ", the average error is:" 1.3199220219646431e-05
time 133 is done ", the average error is:" 1.2409628752458131e-05
time 134 is done ", the average error is:" 1.1667271039964187e-05
time 135 is done ", the average error is:" 1.096932334977e-05
time 136 is done ", the average error is:" 1.0313125861880011e-05
time 137 is done ", the average error is:" 9.696184547354678e-06
time 138 is done ", the average error is:" 9.11615004719731e-06
time 139 is done ", the average error is:" 8.570810635786173e-06
time 140 is done ", the average error is:" 8.058096690063948e-06
time 141 is done ", the average error is:" 7.5760519536999625e-06
time 142 is done ", the average error is:" 7.122844926310988e-06
time 143 is done ", the average error is:" 6.69675009373852e-06
time 144 is done ", the average error is:" 6.29614397656037e-06
time 145 is done ", the 

time 256 is done ", the average error is:" 6.29005233781202e-09
time 257 is done ", the average error is:" 5.913776286956578e-09
time 258 is done ", the average error is:" 5.560006515078011e-09
time 259 is done ", the average error is:" 5.227401848497247e-09
time 260 is done ", the average error is:" 4.9146935501194e-09
time 261 is done ", the average error is:" 4.6206913438782e-09
time 262 is done ", the average error is:" 4.344276940450661e-09
time 263 is done ", the average error is:" 4.084397860132117e-09
time 264 is done ", the average error is:" 3.840064508397882e-09
time 265 is done ", the average error is:" 3.610348031157962e-09
time 266 is done ", the average error is:" 3.3943738850419306e-09
time 267 is done ", the average error is:" 3.1913190319671977e-09
time 268 is done ", the average error is:" 3.000410439601396e-09
time 269 is done ", the average error is:" 2.8209229413342623e-09
time 270 is done ", the average error is:" 2.652172242087993e-09
time 271 is done ", the ave

KeyboardInterrupt: 

# Add layer

In [ ]:
testing_the_model(delta_x=1/20, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_20_20 iteration=1 layers=2 xx", error_file='./prediction/error pairs_20_20 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_20_20 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/20, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_20_40 iteration=1 layers=2 xx", error_file='./prediction/error pairs_20_40 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_20_40 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/20, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_20_80 iteration=1 layers=2 xx", error_file='./prediction/error pairs_20_80 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_20_80 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/20, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_20_160 iteration=1 layers=2 xx", error_file='./prediction/error pairs_20_160 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_20_160 iteration=1 layers=2 xx')

In [ ]:
testing_the_model(delta_x=1/20, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_40_20 iteration=1 layers=2 xx", error_file='./prediction/error pairs_40_20 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_40_20 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/20, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_40_40 iteration=1 layers=2 xx", error_file='./prediction/error pairs_40_40 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_40_40 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/20, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_40_80 iteration=1 layers=2 xx", error_file='./prediction/error pairs_40_80 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_40_80 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/20, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_40_160 iteration=1 layers=2 xx", error_file='./prediction/error pairs_40_160 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_40_160 iteration=1 layers=2 xx')

In [ ]:
testing_the_model(delta_x=1/80, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_80_20 iteration=1 layers=2 xx", error_file='./prediction/error pairs_80_20 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_80_20 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/80, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_80_40 iteration=1 layers=2 xx", error_file='./prediction/error pairs_80_40 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_80_40 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/80, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_80_80 iteration=1 layers=2 xx", error_file='./prediction/error pairs_80_80 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_80_80 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/80, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_80_160 iteration=1 layers=2 xx", error_file='./prediction/error pairs_80_160 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_80_160 iteration=1 layers=2 xx')

In [ ]:
testing_the_model(delta_x=1/160, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_160_20 iteration=1 layers=2 xx", error_file='./prediction/error pairs_160_20 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_160_20 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/160, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_160_40 iteration=1 layers=2 xx", error_file='./prediction/error pairs_160_40 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_160_40 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/160, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_160_80 iteration=1 layers=2 xx", error_file='./prediction/error pairs_160_80 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_160_80 iteration=1 layers=2 xx')
testing_the_model(delta_x=1/160, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, twolayers=True,
                      model_name="./model/model pairs_160_160 iteration=1 layers=2 xx", error_file='./prediction/error pairs_160_160 iteration=1 layers=2 xx', prediction_file='./prediction/prediction pairs_160_160 iteration=1 layers=2 xx')

# Add iteration

In [ ]:
testing_the_model(delta_x=1/20, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_20_20 iteration=2 layers=1 xx", error_file='./prediction/error pairs_20_20 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_20_20 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_20_40 iteration=2 layers=1 xx", error_file='./prediction/error pairs_20_40 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_20_40 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_20_80 iteration=2 layers=1 xx", error_file='./prediction/error pairs_20_80 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_20_80 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_20_160 iteration=2 layers=1 xx", error_file='./prediction/error pairs_20_160 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_20_160 iteration=2 layers=1 xx')

In [ ]:
testing_the_model(delta_x=1/20, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_40_20 iteration=2 layers=1 xx", error_file='./prediction/error pairs_40_20 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_40_20 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_40_40 iteration=2 layers=1 xx", error_file='./prediction/error pairs_40_40 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_40_40 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_40_80 iteration=2 layers=1 xx", error_file='./prediction/error pairs_40_80 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_40_80 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/20, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_40_160 iteration=2 layers=1 xx", error_file='./prediction/error pairs_40_160 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_40_160 iteration=2 layers=1 xx')

In [ ]:
testing_the_model(delta_x=1/80, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_80_20 iteration=2 layers=1 xx", error_file='./prediction/error pairs_80_20 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_80_20 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/80, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_80_40 iteration=2 layers=1 xx", error_file='./prediction/error pairs_80_40 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_80_40 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/80, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_80_80 iteration=2 layers=1 xx", error_file='./prediction/error pairs_80_80 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_80_80 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/80, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_80_160 iteration=2 layers=1 xx", error_file='./prediction/error pairs_80_160 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_80_160 iteration=2 layers=1 xx')

In [ ]:
testing_the_model(delta_x=1/160, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_160_20 iteration=2 layers=1 xx", error_file='./prediction/error pairs_160_20 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_160_20 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/160, delta_t=1/40, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_160_40 iteration=2 layers=1 xx", error_file='./prediction/error pairs_160_40 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_160_40 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/160, delta_t=1/80, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_160_80 iteration=2 layers=1 xx", error_file='./prediction/error pairs_160_80 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_160_80 iteration=2 layers=1 xx')
testing_the_model(delta_x=1/160, delta_t=1/160, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      model_name="./model/model pairs_160_160 iteration=2 layers=1 xx", error_file='./prediction/error pairs_160_160 iteration=2 layers=1 xx', prediction_file='./prediction/prediction pairs_160_160 iteration=2 layers=1 xx')